<h2>Analytics Practicum I - 1st Assignment 2021</h2>
<h3>Exploring the FEC 2016 Elections Data</h3>

---
> Georgia Vlassi p2822001<br />
> Business Anlytics <br />
> Athens University of Economics and Business <br/>

---

The main scope of this assignment is to investigate data provided from Federal Election Commission for the American  elections of 2016.Through different questions, we will investigate and analyze the amounts raised and spent regarding the two main presidential candidates H.Clinton and D.Trump. 






---
## Questions

<h3>1. Identify the top 5 Political Action Committees (PACs), or rather, super-PACs, that supported each of the two presidential candidates, giving the amount of money raised and spent by each one of them.</h3>


---

Before analyzing our data, you should import the following packages.


In [175]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D

from collections import defaultdict
from matplotlib.colors import rgb2hex, Normalize
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase

pd.options.display.float_format = '{:,.2f}'.format

%matplotlib inline
matplotlib.style.use('ggplot')
pd.set_option("display.max_columns", None)



 * The first dataset, which will be used is the contributions from commitees to cadidates and independent expenditure for 2016. You can download both header file and data from https://www.fec.gov/files/bulk-downloads/data_dictionaries/pas2_header_file.csv and https://www.fec.gov/files/bulk-downloads/2016/pas216.zip respectively.
 * The file contains each contribution or independent expenditure made by a PAC, party committee, candidate committee, or other federal committee to a candidate during the two-year election cycle.



In [2]:
pas_headers = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/pas2_header_file.csv')
pas_headers = pas_headers.columns.tolist()
pas_headers

['CMTE_ID',
 'AMNDT_IND',
 'RPT_TP',
 'TRANSACTION_PGI',
 'IMAGE_NUM',
 'TRANSACTION_TP',
 'ENTITY_TP',
 'NAME',
 'CITY',
 'STATE',
 'ZIP_CODE',
 'EMPLOYER',
 'OCCUPATION',
 'TRANSACTION_DT',
 'TRANSACTION_AMT',
 'OTHER_ID',
 'CAND_ID',
 'TRAN_ID',
 'FILE_NUM',
 'MEMO_CD',
 'MEMO_TEXT',
 'SUB_ID']

* Before we import the data, we should convert the field TRANSACTION_AMT to float and all the other fields to strings.

In [3]:
data_types = {header: np.str for header in pas_headers}
data_types['TRANSACTION_AMT'] = np.float

* Full data of file contributions from commitees can be read

In [4]:
pas_contrib = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/pas216.zip', 
                  sep="|", 
                  index_col=False,
                  low_memory=False,
                  names=pas_headers,
                  dtype=data_types)
pas_contrib['TRANSACTION_DT']= pd.to_datetime(pas_contrib['TRANSACTION_DT'], format='%m%d%Y')
print(pas_contrib.shape)
pas_contrib.head(5)

(516394, 22)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00548198,N,M3,G2016,15950887602,24K,CCM,BLAINE FOR CONGRESS,JEFFERSON CITY,MO,65102,NaN,NaN,2015-02-10,"2,500.00",C00458679,H8MO09153,6783511,998835,NaN,NaN,4032020151240895091
1,C00235739,N,M3,P2014,15950887798,24K,CCM,DAN NEWHOUSE FOR CONGRESS,YAKIMA,WA,98909,NaN,NaN,2015-02-05,"2,500.00",C00559393,H4WA04104,B544444,998836,NaN,NaN,4032020151240895118
2,C00235739,N,M3,P2016,15950887799,24K,CCM,CATHY MCMORRIS RODGERS FOR CONGRESS,SPOKANE,WA,99210,NaN,NaN,2015-02-18,"1,000.00",C00390476,H4WA05077,B544838,998836,NaN,NaN,4032020151240895120
3,C00235739,N,M3,G2018,15950887799,24K,CCM,MANCHIN FOR WEST VIRGINIA,WASHINGTON,DC,20002,NaN,NaN,2015-02-05,"1,000.00",C00486563,S0WV00090,B544441,998836,NaN,NaN,4032020151240895121
4,C00235739,N,M3,P2016,15950887790,24K,CCM,PETE AGUILAR FOR CONGRESS,WASHINGTON,DC,20003,NaN,NaN,2015-02-18,"1,500.00",C00510461,H2CA31125,B544833,998836,NaN,NaN,4032020151240895094


Regarding the documentation https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/, we should take into account:
   * The transaction type 24A, which represents an independent expenditure opposing election of candidate.
   * The transaction type 24E, which represents an independent expenditure advocating the election of candidate.
    
We should check if these types exist in our data and then filter:   


In [5]:
pas_contrib['TRANSACTION_TP'].unique()

array(['24K', '24Z', '24E', '24A', '24R', '24C', '24F', '24N'],
      dtype=object)

In [6]:
pas_contrib_up = pas_contrib[pas_contrib['TRANSACTION_TP'].isin(['24E', '24A'])]
print(pas_contrib_up.shape)
pas_contrib_up.head(5)

(239515, 22)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
802,C00365296,N,Q1,P2015,15970350002,24E,ORG,TERRA STRATEGIES LLC,DES MOINES,IA,50309,NaN,NaN,2015-03-24,"12,404.00",H6MO05171,H6MO05171,SE.22416,1003479,NaN,NaN,4041620151241906737
2398,C00520684,N,MY,G2014,201507309000454990,24A,ORG,ODNEY ADVERTISING AGENCY,BISMARCK,ND,58502,NaN,NaN,2015-01-08,"2,195.00",H4ND00046,H4ND00046,SE.4381,1018960,NaN,NaN,4073020151248026313
3620,C00570739,N,MY,P2016,201507319000476060,24E,PAC,MAKE DC LISTEN,ATHENS,GA,30605,NaN,NaN,2015-04-20,61.00,P60006111,P60006111,EC0B8BFA8422A4D8BABD,1019398,NaN,NaN,4073120151248081657
3621,C00570739,N,MY,P2016,201507319000476060,24E,ORG,FACEBOOK,MENLO PARK,CA,940251452,NaN,NaN,2015-04-21,325.00,P60006111,P60006111,E04A7B32ADF0840C5889,1019398,NaN,NaN,4073120151248081659
3622,C00570739,N,MY,P2016,201507319000476061,24E,ORG,ENVISION MARKETING,LYNCHBURG,VA,245024202,NaN,NaN,2015-04-17,"25,532.00",P60006111,P60006111,E2840F8EF99584536A12,1019398,NaN,NaN,4073120151248081661


In [7]:
pas_contrib['TRANSACTION_TP'].unique()

array(['24K', '24Z', '24E', '24A', '24R', '24C', '24F', '24N'],
      dtype=object)

* The second dataset, which will be used is the candidate master for 2016. Υou can download both header file and data https://www.fec.gov/files/bulk-downloads/data_dictionaries/cn_header_file.csv and https://www.fec.gov/files/bulk-downloads/2016/cn16.zip

* The candidate master file contains one record for each candidate who has either registered with the Federal Election Commission. We want to get the candidate name of each candidate id.


In [8]:
cn_headers =  pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/cn_header_file.csv')
cn_headers = cn_headers.columns.tolist()
cn_headers

['CAND_ID',
 'CAND_NAME',
 'CAND_PTY_AFFILIATION',
 'CAND_ELECTION_YR',
 'CAND_OFFICE_ST',
 'CAND_OFFICE',
 'CAND_OFFICE_DISTRICT',
 'CAND_ICI',
 'CAND_STATUS',
 'CAND_PCC',
 'CAND_ST1',
 'CAND_ST2',
 'CAND_CITY',
 'CAND_ST',
 'CAND_ZIP']

*  Full data of file candidates can be read

In [9]:
candidates = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/cn16.zip', 
                  sep="|", 
                  index_col=False,
                  low_memory=False,
                  names=cn_headers)
print(candidates.shape)
candidates.head(5)

(7396, 15)


,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
0,H0AK00097,"COX, JOHN R.",REP,2014,AK,H,0.00,C,N,C00525261,P.O. BOX 1092,NaN,ANCHOR POINT,AK,"99,556.00"
1,H0AL02087,"ROBY, MARTHA",REP,2016,AL,H,2.00,I,C,C00462143,PO BOX 195,NaN,MONTGOMERY,AL,"36,101.00"
2,H0AL02095,"JOHN, ROBERT E JR",IND,2016,AL,H,2.00,C,N,NaN,1465 W OVERBROOK RD,NaN,MILLBROOK,AL,"36,054.00"
3,H0AL05049,"CRAMER, ROBERT E ""BUD"" JR",DEM,2008,AL,H,5.00,NaN,P,C00239038,PO BOX 2621,NaN,HUNTSVILLE,AL,"35,804.00"
4,H0AL05163,"BROOKS, MO",REP,2016,AL,H,5.00,I,C,C00464149,7610 FOXFIRE DRIVE,NaN,HUNTSVILLE,AL,"35,802.00"


* From file candidates we will only keep the following:
    * The candidates running for president in the 2016 elections.
    * The value 'P' for the column CAND_OFFICE means that the candidate runs for president.
    * The value 'C' for the column CAND_STATUS means 'statutory candidate'
    * Fields 'CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_OFFICE', 'CAND_STATUS'


In [10]:
candidates = candidates.loc[(candidates['CAND_ELECTION_YR'] == 2016) & (candidates['CAND_OFFICE'] == 'P') & (candidates['CAND_STATUS'] == 'C')]
candidates = candidates[['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_OFFICE', 'CAND_STATUS']]
print(candidates.shape)
candidates.sample(5)

(72, 6)


,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE,CAND_STATUS
5298,P60012234,"JOHNSON, JOHN FITZGERALD MR.",IND,2016,P,C
4985,P60008398,"JINDAL, BOBBY",REP,2016,P,C
5640,P60016383,"KITTINGTON, VALMA",REP,2016,P,C
4693,P60005279,"BOWERS, KERRY DALE",REP,2016,P,C
4763,P60006046,"WALKER, SCOTT",REP,2016,P,C


In [11]:
candidates['CAND_NAME'].unique()

array(['CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE',
       'SCHRINER, JOSEPH CHARLES', 'BROWN, HARLEY D',
       'BICKELMEYER, MICHAEL', 'JOHNSON, GARY / WILLIAM "BILL" WELD',
       'SANTORUM, RICHARD J.', 'HILL, CHRISTOPHER V',
       'PERRY, JAMES R (RICK)', 'STEIN, JILL', 'WELLS, ROBERT CARR JR',
       'WHITE, JEROME S', 'KREML, WILLIAM P', 'PAUL, RAND',
       'KASICH, JOHN R', 'MOOREHEAD, MONICA GAIL',
       'ADESHINA, YINKA ABOSEDE', 'JEROBOAN, KINS',
       'STEINBERG, MICHAEL ALAN', 'LOWER, BARTHOLOMEW JAMES MR.',
       'BOWERS, KERRY DALE', 'SHERMAN, JEFFERSON WOODSON',
       'CHRISTENSEN, DALE H', 'DUCKWALD, WANDA GAYLE',
       'CARSON, BENJAMIN S SR MD', 'EVERSON, MARK', 'WALKER, SCOTT',
       'CRUZ, RAFAEL EDWARD "TED"', 'LYNCH, DENNIS MICHAEL',
       'SANDERS, BERNARD', 'FIORINA, CARLY', 'KELSO, LLOYD THOMAS',
       'SCROGGIE, JEREMY', 'WILSON, WILLIE', 'PATAKI, GEORGE E',
       "O'MALLEY, MARTIN JOSEPH", 'GRAHAM, LINDSEY O',
       'WINSLOW, BRAD MR.', 'MA

Clear values of H.Clinton and D.Trump

In [12]:
candidates.loc[candidates['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
candidates.loc[candidates['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

In [13]:
candidates = candidates.loc[candidates['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])]
print(candidates.shape)
candidates.head()

(2, 6)


,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE,CAND_STATUS
4367,P00003392,"CLINTON, HILLARY RODHAM",DEM,2016,P,C
6298,P80001571,"TRUMP, DONALD J.",REP,2016,P,C


* We will merge the 'pas_contrib' file with 'candidates' file

In [14]:
contrib_candidates = pd.merge(pas_contrib_up, candidates[['CAND_ID', 'CAND_NAME','CAND_PTY_AFFILIATION',]], on='CAND_ID', how='inner')
print(contrib_candidates.shape)
contrib_candidates.head(5)

(109630, 24)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME,CAND_PTY_AFFILIATION
0,C90015454,N,Q3,P2016,201508119000813972,24E,IND,"MARLOWE, MARK ANTHONY",IOWA CITY,IA,52244,NaN,NaN,2015-07-04,"2,250.00",P80001571,P80001571,F57.000001,1021308,NaN,NaN,4081220151248549144,"TRUMP, DONALD J.",REP
1,C00004036,N,YE,P2016,201601309004951347,24A,ORG,FACEBOOK ADVERTISING,CHICAGO,IL,60693,NaN,NaN,2015-12-15,"3,698.00",P80001571,P80001571,D362280,1046003,X,NaN,4021020161262352096,"TRUMP, DONALD J.",REP
2,C00004036,N,YE,P2016,201601309004951349,24A,ORG,RISING TIDE INTERACTIVE LLC,WASHINGTON,DC,20015,NaN,NaN,2015-12-15,"1,240.00",P80001571,P80001571,D362284,1046003,X,NaN,4021020161262352100,"TRUMP, DONALD J.",REP
3,C00004036,N,YE,P2016,201601309004951350,24A,ORG,GOOGLE,MOUNTAIN VIEW,CA,94043,NaN,NaN,2015-12-15,"3,698.00",P80001571,P80001571,D362290,1046003,X,NaN,4021020161262352102,"TRUMP, DONALD J.",REP
4,C00004036,N,YE,P2016,201601309004951352,24A,ORG,GOOGLE,MOUNTAIN VIEW,CA,94043,NaN,NaN,2015-12-22,48.00,P80001571,P80001571,D362890,1046003,NaN,NaN,4021020161262352106,"TRUMP, DONALD J.",REP


* The third dataset, which will be used is the PAC summary for 2016. Υou can download the data from https://www.fec.gov/files/bulk-downloads/2016/webk16.zip

* This file gives overall receipts and disbursements for each PAC and party committee registered with the commission, along with a breakdown of overall receipts by source and totals for contributions to other committees, independent expenditures made and other information.

In [15]:
pac_headers = ['CMTE_ID', 'CMTE_NM', 'CMTE_TP', 'CMTE_DSGN', 'CMTE_FILING_FREQ', 'TTL_RECEIPTS', 'TRANS_FROM_AFF', 'INDV_CONTRIB',
           'OTHER_POL_CMTE_CONTRIB', 'CAND_CONTRIB', 'CAND_LOANS', 'TTL_LOANS_RECEIVED', 'TTL_DISB', 'TRANF_TO_AFF', 
           'INDV_REFUNDS', 'OTHER_POL_CMTE_REFUNDS', 'CAND_LOAN_REPAY', 'LOAN_REPAY', 'COH_BOP', 'COH_COP', 'DEBTS_OWED_BY',
           'NONFED_TRANS_RECEIVED', 'CONTRIB_TO_OTHER_CMTE', 'IND_EXP', 'PTY_COORD_EXP', 'NONFED_SHARE_EXP', 'CVG_END_DT']
pac_headers

['CMTE_ID',
 'CMTE_NM',
 'CMTE_TP',
 'CMTE_DSGN',
 'CMTE_FILING_FREQ',
 'TTL_RECEIPTS',
 'TRANS_FROM_AFF',
 'INDV_CONTRIB',
 'OTHER_POL_CMTE_CONTRIB',
 'CAND_CONTRIB',
 'CAND_LOANS',
 'TTL_LOANS_RECEIVED',
 'TTL_DISB',
 'TRANF_TO_AFF',
 'INDV_REFUNDS',
 'OTHER_POL_CMTE_REFUNDS',
 'CAND_LOAN_REPAY',
 'LOAN_REPAY',
 'COH_BOP',
 'COH_COP',
 'DEBTS_OWED_BY',
 'NONFED_TRANS_RECEIVED',
 'CONTRIB_TO_OTHER_CMTE',
 'IND_EXP',
 'PTY_COORD_EXP',
 'NONFED_SHARE_EXP',
 'CVG_END_DT']

In [16]:
pac_sum = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/webk16.zip', sep="|", index_col=False, names= pac_headers)
print(pac_sum.shape)
pac_sum.head(5)

(12048, 27)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,CAND_LOANS,TTL_LOANS_RECEIVED,TTL_DISB,TRANF_TO_AFF,INDV_REFUNDS,OTHER_POL_CMTE_REFUNDS,CAND_LOAN_REPAY,LOAN_REPAY,COH_BOP,COH_COP,DEBTS_OWED_BY,NONFED_TRANS_RECEIVED,CONTRIB_TO_OTHER_CMTE,IND_EXP,PTY_COORD_EXP,NONFED_SHARE_EXP,CVG_END_DT
0,C00000059,HALLMARK CARDS PAC,Q,U,M,"123,198.92",0.00,"123,198.92",0.00,0.00,0.00,0.00,"88,500.00",0.00,0.00,0.00,0.00,0.00,"104,794.36","139,493.28",0.00,0.00,"88,500.00",0.00,0.00,0.00,12/31/2016
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,Q,B,M,"2,114,478.16",0.00,"2,099,958.16",0.00,0.00,0.00,0.00,"2,047,839.79","1,790.00","12,621.71",0.00,0.00,0.00,"552,464.38","619,102.75",0.00,0.00,"1,853,000.00","141,616.35",0.00,0.00,12/31/2016
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,N,U,Q,"41,455.17","41,453.00",0.00,0.00,0.00,0.00,0.00,"39,672.85",0.00,0.00,0.00,0.00,0.00,192.00,"1,974.00",0.00,0.00,0.00,0.00,0.00,0.00,12/31/2016
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,Q,U,Q,"19,065.00",0.00,"19,065.00",0.00,0.00,0.00,0.00,"17,592.50","2,592.50",0.00,0.00,0.00,0.00,"4,681.26","6,153.76",0.00,0.00,"15,000.00",0.00,0.00,0.00,12/31/2016
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,Q,U,Q,"143,170.00",0.00,"142,570.00",0.00,0.00,0.00,0.00,"85,918.48","13,222.50",0.00,0.00,0.00,0.00,"35,530.91","92,782.43",0.00,0.00,"3,000.00",0.00,0.00,0.00,12/31/2016


* We will merge the 'contrib_candidates' file with 'pac_sum'

In [17]:
final = pd.merge(pac_sum, contrib_candidates, on='CMTE_ID', how='inner')
print(final.shape)
final.head(5)

(109616, 50)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,CAND_LOANS,TTL_LOANS_RECEIVED,TTL_DISB,TRANF_TO_AFF,INDV_REFUNDS,OTHER_POL_CMTE_REFUNDS,CAND_LOAN_REPAY,LOAN_REPAY,COH_BOP,COH_COP,DEBTS_OWED_BY,NONFED_TRANS_RECEIVED,CONTRIB_TO_OTHER_CMTE,IND_EXP,PTY_COORD_EXP,NONFED_SHARE_EXP,CVG_END_DT,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME,CAND_PTY_AFFILIATION
0,C00000935,DCCC,Y,U,M,"220,891,388.49","6,619,252.47","147,346,246.71","38,631,410.54",0.00,0.00,0.00,"216,358,583.53","26,206,817.84","2,732,586.69","14,300.00",0.00,"13,000,000.00","2,149,438.76","6,682,243.72","14,000,000.00",0.00,"403,810.19","80,378,630.35","3,612,999.17",0.00,12/31/2016,A,M10,G2016,201706169056600064,24A,ORG,"MOORE CAMPAIGNS, LLC",WASHINGTON,DC,20010,NaN,NaN,2016-09-28,"2,168.00",P80001571,P80001571,SE-950758,1166132,NaN,DATE OF DISSEMINATION: 09/29/16,4061620171409944958,"TRUMP, DONALD J.",REP
1,C00000935,DCCC,Y,U,M,"220,891,388.49","6,619,252.47","147,346,246.71","38,631,410.54",0.00,0.00,0.00,"216,358,583.53","26,206,817.84","2,732,586.69","14,300.00",0.00,"13,000,000.00","2,149,438.76","6,682,243.72","14,000,000.00",0.00,"403,810.19","80,378,630.35","3,612,999.17",0.00,12/31/2016,A,M10,G2016,201706169056600065,24A,ORG,"MOORE CAMPAIGNS, LLC",WASHINGTON,DC,20010,NaN,NaN,2016-09-28,836.00,P80001571,P80001571,SE-950759,1166132,NaN,DATE OF DISSEMINATION: 09/29/16,4061620171409944960,"TRUMP, DONALD J.",REP
2,C00000935,DCCC,Y,U,M,"220,891,388.49","6,619,252.47","147,346,246.71","38,631,410.54",0.00,0.00,0.00,"216,358,583.53","26,206,817.84","2,732,586.69","14,300.00",0.00,"13,000,000.00","2,149,438.76","6,682,243.72","14,000,000.00",0.00,"403,810.19","80,378,630.35","3,612,999.17",0.00,12/31/2016,A,M10,G2016,201706169056600075,24A,ORG,GREAT AMERICAN MEDIA,WASHINGTON,DC,20007,NaN,NaN,2016-09-29,"77,062.00",P80001571,P80001571,SE-951093,1166132,NaN,DATE OF DISSEMINATION: 10/04/16,4061620171409945000,"TRUMP, DONALD J.",REP
3,C00000935,DCCC,Y,U,M,"220,891,388.49","6,619,252.47","147,346,246.71","38,631,410.54",0.00,0.00,0.00,"216,358,583.53","26,206,817.84","2,732,586.69","14,300.00",0.00,"13,000,000.00","2,149,438.76","6,682,243.72","14,000,000.00",0.00,"403,810.19","80,378,630.35","3,612,999.17",0.00,12/31/2016,A,M10,G2016,201706169056600084,24A,ORG,GREAT AMERICAN MEDIA,WASHINGTON,DC,20007,NaN,NaN,2016-09-29,"108,354.00",P80001571,P80001571,SE-950803,1166132,NaN,DATE OF DISSEMINATION: 10/04/16,4061620171409945035,"TRUMP, DONALD J.",REP
4,C00000935,DCCC,Y,U,M,"220,891,388.49","6,619,252.47","147,346,246.71","38,631,410.54",0.00,0.00,0.00,"216,358,583.53","26,206,817.84","2,732,586.69","14,300.00",0.00,"13,000,000.00","2,149,438.76","6,682,243.72","14,000,000.00",0.00,"403,810.19","80,378,630.35","3,612,999.17",0.00,12/31/2016,A,M10,G2016,201706169056600086,24A,ORG,"RALSTON LAPP MEDIA, LLC",WASHINGTON,DC,20007,NaN,NaN,2016-09-30,"6,220.00",P80001571,P80001571,SE-950901,1166132,NaN,DATE OF DISSEMINATION: 10/01/16,4061620171409945045,"TRUMP, DONALD J.",REP


*** Check the below ***

In [18]:
final['CAND_NAME'].unique()

array(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'], dtype=object)

In [19]:
final['TRANSACTION_TP'].unique()

array(['24A', '24E'], dtype=object)

For better perfomance some columns must be excluded from our final dataset. Regarding the 1st question the following columns will make up the final dataset: 

In [20]:
final = final[['CMTE_ID','CMTE_NM','CMTE_TP','TTL_RECEIPTS','OTHER_POL_CMTE_CONTRIB','CAND_CONTRIB','IND_EXP','CAND_ID','CAND_NAME','CAND_PTY_AFFILIATION','ENTITY_TP','NAME','SUB_ID','TRANSACTION_AMT','TRANSACTION_DT','TRANSACTION_PGI','TRANSACTION_TP']]
print(final.shape)
final.sample(5)

(109616, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
52908,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,IND,"RUIZ, LUZMEILYN",4101320161340955502,47.00,2016-09-27,G2016,24E
66333,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,IND,"GOIDICH, ALYSSA",4101320161340950735,17.00,2016-09-23,G2016,24E
66803,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,SUPERIOR BP,4101320161340927727,5.00,2016-09-01,G2016,24E
61259,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,BUDGET-PITTSBURGH,4101320161340929836,10.00,2016-09-06,G2016,24E
32656,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P80001571,"TRUMP, DONALD J.",REP,ORG,CANDLEWOOD,4020920171370075482,9.00,2016-08-29,G2016,24A


* Our dataset includes both contributions and refunds.

In [21]:
(final['TRANSACTION_AMT'] > 0).value_counts()

True     109258
False       358
Name: TRANSACTION_AMT, dtype: int64

* We will keep only positive contributions.

In [23]:
final = final[final['TRANSACTION_AMT'] > 0]

* As our main scope is to find the top 5 super PACs, we will filter our data with CMTE_TP = 'O'.  The 'O' indicates the Independent expenditure-only (Super PACs) as shown in documentation https://www.fec.gov/campaign-finance-data/committee-type-code-descriptions/

In [24]:
final_o = final.loc[final['CMTE_TP'].isin(['O'])]
print(final_o.shape)
final_o.sample(5)

(6245, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
8093,C00587022,COURAGEOUS CONSERVATIVES PAC,O,"424,500.21","2,500.00",0.00,"324,961.94",P80001571,"TRUMP, DONALD J.",REP,ORG,MOUNTAINTOP MEDIA,4052020161292980432,250.00,2016-04-29,P2016,24A
108377,C00489799,PLANNED PARENTHOOD VOTES,O,"22,266,898.24","3,712,020.00",0.00,"12,628,454.91",P80001571,"TRUMP, DONALD J.",REP,ORG,MOXIE MEDIA INC.,4031720171380310471,"5,000.00",2016-08-24,G2016,24A
107631,C00495861,PRIORITIES USA ACTION,O,"192,065,767.58","30,083,034.37",0.00,"133,408,056.41",P80001571,"TRUMP, DONALD J.",REP,ORG,"RALSTON LAPP MEDIA, LLC",4111520161347283193,"32,121.00",2016-06-19,G2016,24A
108861,C00487470,CLUB FOR GROWTH ACTION,O,"19,936,801.97","4,661.11",0.00,"19,182,422.19",P80001571,"TRUMP, DONALD J.",REP,ORG,CLUB FOR GROWTH,4112020151257319032,39.00,2015-10-22,P2016,24A
2815,C00497420,CITIZENS UNITED SUPER PAC LLC,O,"1,142,857.09",0.00,0.00,"955,345.69",P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,HSP DIRECT LLC,4080420171442569331,"5,984.00",2016-06-13,G2016,24A


<h3>Hillary Clinton </h3>

In [25]:
adv_clinton = final.loc[final['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(adv_clinton.shape)
adv_clinton.head(5)

(64144, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
183,C00002089,COMMUNICATIONS WORKERS OF AMERICA-COPE POLITIC...,Q,"7,930,523.68",0.00,0.00,150.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,OFFBEAT PRESS,4020620171369713210,150.00,2016-09-26,G2016,24E
184,C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,"12,747,221.51",0.00,0.00,"180,480.13",P00003392,"CLINTON, HILLARY RODHAM",DEM,NaN,JOE TRIPPI AND ASSOCIATES INC.,4082320161313244264,746.00,2016-07-26,P2016,24E
185,C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,"12,747,221.51",0.00,0.00,"180,480.13",P00003392,"CLINTON, HILLARY RODHAM",DEM,NaN,JOE TRIPPI AND ASSOCIATES INC.,4082320161313244267,"1,702.00",2016-07-28,P2016,24E
186,C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,"12,747,221.51",0.00,0.00,"180,480.13",P00003392,"CLINTON, HILLARY RODHAM",DEM,NaN,JOE TRIPPI AND ASSOCIATES INC.,4082320161313244269,"6,041.00",2016-07-28,P2016,24E
187,C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,"12,747,221.51",0.00,0.00,"180,480.13",P00003392,"CLINTON, HILLARY RODHAM",DEM,NaN,JOE TRIPPI AND ASSOCIATES INC.,4082320161313244271,"124,686.00",2016-07-28,P2016,24E


* Find commitees advocating H.Clinton

In [26]:
adv_clinton = adv_clinton.loc[adv_clinton['TRANSACTION_TP'].isin(['24Ε'])]
adv_clinton = adv_clinton[['CAND_NAME', 'CMTE_NM', 'CMTE_TP','TRANSACTION_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(adv_clinton.shape)
adv_clinton.head(5)

(0, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME


In [27]:
opp_clinton = final.loc[final['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(opp_clinton.shape)
opp_clinton.head(5)

(45114, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
0,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"MOORE CAMPAIGNS, LLC",4061620171409944958,"2,168.00",2016-09-28,G2016,24A
1,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"MOORE CAMPAIGNS, LLC",4061620171409944960,836.00,2016-09-28,G2016,24A
2,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,GREAT AMERICAN MEDIA,4061620171409945000,"77,062.00",2016-09-29,G2016,24A
3,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,GREAT AMERICAN MEDIA,4061620171409945035,"108,354.00",2016-09-29,G2016,24A
4,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"RALSTON LAPP MEDIA, LLC",4061620171409945045,"6,220.00",2016-09-30,G2016,24A


* Find committes oppposing H.Clinton

In [28]:
opp_clinton = opp_clinton.loc[opp_clinton['TRANSACTION_TP'].isin(['24A'])]
opp_clinton = opp_clinton[['CAND_NAME', 'CMTE_NM', 'CMTE_TP','TRANSACTION_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(opp_clinton.shape)
opp_clinton.head(5)

(43823, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
0,"TRUMP, DONALD J.",DCCC,Y,24A,"2,168.00","220,891,388.49","80,378,630.35","MOORE CAMPAIGNS, LLC"
1,"TRUMP, DONALD J.",DCCC,Y,24A,836.00,"220,891,388.49","80,378,630.35","MOORE CAMPAIGNS, LLC"
2,"TRUMP, DONALD J.",DCCC,Y,24A,"77,062.00","220,891,388.49","80,378,630.35",GREAT AMERICAN MEDIA
3,"TRUMP, DONALD J.",DCCC,Y,24A,"108,354.00","220,891,388.49","80,378,630.35",GREAT AMERICAN MEDIA
4,"TRUMP, DONALD J.",DCCC,Y,24A,"6,220.00","220,891,388.49","80,378,630.35","RALSTON LAPP MEDIA, LLC"


* Merge these two datasets for H.Clinton

In [29]:
super_clinton = pd.concat([adv_clinton, opp_clinton])
print(super_clinton.shape)
super_clinton.head()

(43823, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
0,"TRUMP, DONALD J.",DCCC,Y,24A,"2,168.00","220,891,388.49","80,378,630.35","MOORE CAMPAIGNS, LLC"
1,"TRUMP, DONALD J.",DCCC,Y,24A,836.00,"220,891,388.49","80,378,630.35","MOORE CAMPAIGNS, LLC"
2,"TRUMP, DONALD J.",DCCC,Y,24A,"77,062.00","220,891,388.49","80,378,630.35",GREAT AMERICAN MEDIA
3,"TRUMP, DONALD J.",DCCC,Y,24A,"108,354.00","220,891,388.49","80,378,630.35",GREAT AMERICAN MEDIA
4,"TRUMP, DONALD J.",DCCC,Y,24A,"6,220.00","220,891,388.49","80,378,630.35","RALSTON LAPP MEDIA, LLC"


* Keep seperately the super PACs for H.Clinton

In [30]:
super_clinton_o = super_clinton.loc[super_clinton['CMTE_TP'].isin(['O'])].copy()
print(super_clinton_o.shape)
super_clinton_o.head()

(2371, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
2044,"TRUMP, DONALD J.",WOMEN VOTE!,O,24A,"12,333.00","36,685,866.33","33,167,398.37","PRECISION NETWORK, LLC"
2045,"TRUMP, DONALD J.",WOMEN VOTE!,O,24A,"93,219.00","36,685,866.33","33,167,398.37","PRECISION NETWORK, LLC"
2046,"TRUMP, DONALD J.",WOMEN VOTE!,O,24A,"673,224.00","36,685,866.33","33,167,398.37","PRECISION NETWORK, LLC"
2047,"TRUMP, DONALD J.",WOMEN VOTE!,O,24A,"450,000.00","36,685,866.33","33,167,398.37","PRECISION NETWORK, LLC"
2048,"TRUMP, DONALD J.",WOMEN VOTE!,O,24A,"558,000.00","36,685,866.33","33,167,398.37",WATERFRONT STRATEGIES


<h3>Donald Trump </h3>

In [31]:
adv_trump = final.loc[final['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(adv_trump.shape)
adv_trump.head(5)

(45114, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
0,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"MOORE CAMPAIGNS, LLC",4061620171409944958,"2,168.00",2016-09-28,G2016,24A
1,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"MOORE CAMPAIGNS, LLC",4061620171409944960,836.00,2016-09-28,G2016,24A
2,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,GREAT AMERICAN MEDIA,4061620171409945000,"77,062.00",2016-09-29,G2016,24A
3,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,GREAT AMERICAN MEDIA,4061620171409945035,"108,354.00",2016-09-29,G2016,24A
4,C00000935,DCCC,Y,"220,891,388.49","38,631,410.54",0.00,"80,378,630.35",P80001571,"TRUMP, DONALD J.",REP,ORG,"RALSTON LAPP MEDIA, LLC",4061620171409945045,"6,220.00",2016-09-30,G2016,24A


* Find committes advocating for D.Trump

In [32]:
adv_trump = adv_trump.loc[adv_trump['TRANSACTION_TP'].isin(['24E'])]
adv_trump = adv_trump[['CAND_NAME', 'CMTE_NM', 'CMTE_TP','TRANSACTION_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(adv_trump.shape)
adv_trump.sample()

(1291, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
1141,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,145.00,"21,591,111.40","19,241,228.01",PROLIST INC.


In [33]:
opp_trump = final.loc[final['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(opp_trump.shape)
opp_trump.sample(5)

(64144, 17)


,CMTE_ID,CMTE_NM,CMTE_TP,TTL_RECEIPTS,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,IND_EXP,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,ENTITY_TP,NAME,SUB_ID,TRANSACTION_AMT,TRANSACTION_DT,TRANSACTION_PGI,TRANSACTION_TP
66396,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,IND,"YOUNG, JOHN",4101320161340904932,34.00,2016-07-26,G2016,24E
69354,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,BUDGET-CLEVELAND,4020920171370153095,12.00,2016-10-27,G2016,24E
74901,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,BUDGET-CLEVELAND,4020920171370072608,13.00,2016-08-25,G2016,24E
4349,C00544767,STOP HILLARY PAC,V,"6,843,714.67",0.00,0.00,"3,462,965.48",P00003392,"CLINTON, HILLARY RODHAM",DEM,ORG,CAMPAIGN SOLUTIONS,4022220191643428287,"14,375.00",2016-10-13,G2016,24A
66644,C90011156,WORKING AMERICA,I,"11,373,645.00",0.00,0.00,0.00,P00003392,"CLINTON, HILLARY RODHAM",DEM,IND,"WHITTIER, SHAYOLONDA",4101320161340951581,3.00,2016-09-23,G2016,24E


* Find commitees opposing D.Trump

In [34]:
opp_trump = opp_trump.loc[opp_trump['TRANSACTION_TP'].isin(['24A'])]
opp_trump = opp_trump[['CAND_NAME', 'CMTE_NM', 'CMTE_TP','TRANSACTION_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(opp_trump.shape)
opp_trump.sample()

(5393, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
5919,"CLINTON, HILLARY RODHAM",VIGOP (VIRGIN ISLANDS REPUBLICAN PARTY),Q,24A,24.00,"3,493,514.62","737,140.10",DIRECT SUPPORT SERVICES INC


* Merge these two datasets for D.Trump


In [35]:
super_trump = pd.concat([adv_trump, opp_trump])
print(super_trump.shape)
super_trump.head()

(6684, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
344,"TRUMP, DONALD J.",SEIU COPE (SERVICE EMPLOYEES INTERNATIONAL UNI...,Q,24E,"4,580.00","50,264,684.02","8,550,288.58",THE PIVOT GROUP
1068,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,113.00,"21,591,111.40","19,241,228.01",MAHONING COUNTY AGRICULTURAL SOCIETY
1069,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,7.00,"21,591,111.40","19,241,228.01","WISCONSIN FIREARM OWNERS, RANGES, CLUBS AND ED..."
1070,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,205.00,"21,591,111.40","19,241,228.01",KITTITAS VALLEY EVENT CENTER
1071,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,21.00,"21,591,111.40","19,241,228.01","DOWNTOWN EAU CLAIRE, INC."


* Keep seperately the super PACs for D.Trump

In [36]:
super_trump_o = super_trump.loc[super_trump['CMTE_TP'].isin(['O'])].copy()
print(super_trump_o.shape)
super_trump_o.head()

(2000, 8)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
2259,"TRUMP, DONALD J.",TEXAS PATRIOTS PAC,O,24E,86.00,"396,682.92","87,288.87",CONROE COURIER
2349,"TRUMP, DONALD J.",CITIZENS UNITED SUPER PAC LLC,O,24E,"20,000.00","1,142,857.09","955,345.69",INFOCISION
2350,"TRUMP, DONALD J.",CITIZENS UNITED SUPER PAC LLC,O,24E,"10,000.00","1,142,857.09","955,345.69",MDI IMAGING & MAILING
2351,"TRUMP, DONALD J.",CITIZENS UNITED SUPER PAC LLC,O,24E,"17,550.00","1,142,857.09","955,345.69",HSP DIRECT LLC
2352,"TRUMP, DONALD J.",CITIZENS UNITED SUPER PAC LLC,O,24E,773.00,"1,142,857.09","955,345.69",STMP


* We will calculate the total amounts for PACs and super-PACs for H.Clinton

In [37]:
super_clinton_party = super_clinton.groupby(['CMTE_NM']).sum().sort_values(by='IND_EXP', ascending=False)
super_clinton_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
PRIORITIES USA ACTION,"126,062,193.00","32,267,048,953.44","22,412,553,476.88"
DCCC,"6,693,341.00","40,423,124,093.67","14,709,289,354.05"
CLUB FOR GROWTH ACTION,"7,054,203.00","4,884,516,482.65","4,699,693,436.55"
OUR PRINCIPLES PAC,"16,353,117.00","3,900,105,125.25","3,757,059,347.85"
HOUSE MAJORITY PAC,"2,761,357.00","4,190,405,342.25","3,560,282,061.00"


<h4>The TOP 5 PACs and super-PACs for Hillary Clinton</h4>

In [38]:
super_clinton_party_apply = super_clinton.groupby(['CMTE_NM']).apply(lambda x:x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'CMTE_TP',]]
super_clinton_party = super_clinton_party_apply.sort_values(by='IND_EXP', ascending=False)
super_clinton_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,CMTE_TP
CMTE_NM,,,,
PRIORITIES USA ACTION,"10,774,356.00","192,065,767.58","133,408,056.41",O
RIGHT TO RISE USA,"19,411.00","121,695,224.05","86,817,478.31",O
DCCC,"467,549.00","220,891,388.49","80,378,630.35",Y
SENATE MAJORITY PAC,"313,650.00","92,821,080.67","75,413,534.87",O
DSCC,"240,829.00","179,800,228.74","60,421,908.00",Y


* We will calculate the total amounts for PACs and super-PACs for D.Trump

In [39]:
super_trump_party = super_trump.groupby(['CMTE_NM']).sum().sort_values(by='IND_EXP', ascending=False)
super_trump_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
GREAT AMERICA PAC,"23,885,544.00","11,158,298,772.52","9,183,640,334.99"
FUTURE45,"24,219,101.00","5,224,272,688.36","5,071,184,750.83"
NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICAL VICTORY FUND,"9,315,576.00","5,095,502,290.40","4,540,929,810.36"
TEA PARTY MAJORITY FUND,"3,602,443.00","3,687,443,562.00","2,008,823,245.00"
REBUILDING AMERICA NOW,"21,199,098.00","1,440,607,533.34","1,208,873,407.85"


<h4>The TOP 5 PACs and super-PACs for Donald Trump</h4>

In [40]:
super_trump_party_apply = super_trump.groupby(['CMTE_NM']).apply(lambda x:x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'CMTE_TP',]]
super_trump_party = super_trump_party_apply.sort_values(by='IND_EXP', ascending=False)
super_trump_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,CMTE_TP
CMTE_NM,,,,
"FREEDOM PARTNERS ACTION FUND, INC.","13,400.00","28,201,407.05","29,728,850.26",O
FUTURE45,"7,000,000.00","24,996,520.04","24,264,041.87",O
GREAT AMERICA PAC,"3,000,000.00","28,684,572.68","23,608,329.91",W
REBUILDING AMERICA NOW,"1,746,350.00","23,616,516.94","19,817,596.85",O
NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICAL VICTORY FUND,"2,235,223.00","21,591,111.40","19,241,228.01",Q


In order to be more precisely to the 1st question, we should show the TOP 5 super PACS for each candidate

<h4>The TOP 5 super PACs for Hillary Clinton</h4>

In [41]:
super_clinton_party_o = super_clinton_o.groupby(['CMTE_NM']).apply(lambda x:x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'CMTE_TP',]]
superPAC_clinton_party = super_clinton_party_o.sort_values(by='IND_EXP', ascending=False)
superPAC_clinton_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,CMTE_TP
CMTE_NM,,,,
PRIORITIES USA ACTION,"10,774,356.00","192,065,767.58","133,408,056.41",O
RIGHT TO RISE USA,"19,411.00","121,695,224.05","86,817,478.31",O
SENATE MAJORITY PAC,"313,650.00","92,821,080.67","75,413,534.87",O
CONSERVATIVE SOLUTIONS PAC,"1,510,406.00","60,564,219.16","55,443,629.89",O
HOUSE MAJORITY PAC,"539,240.00","55,872,071.23","47,470,427.48",O


<h4>The TOP 5 super PACs for Donald Trump</h4>

In [42]:
super_trump_party_o = super_trump_o.groupby(['CMTE_NM']).apply(lambda x:x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'CMTE_TP',]]
superPAC_trump_party = super_trump_party_o.sort_values(by='IND_EXP', ascending=False)
superPAC_trump_party.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,CMTE_TP
CMTE_NM,,,,
"FREEDOM PARTNERS ACTION FUND, INC.","13,400.00","28,201,407.05","29,728,850.26",O
FUTURE45,"7,000,000.00","24,996,520.04","24,264,041.87",O
REBUILDING AMERICA NOW,"1,746,350.00","23,616,516.94","19,817,596.85",O
CLUB FOR GROWTH ACTION,52.00,"19,936,801.97","19,182,422.19",O
UNITED WE CAN,"4,392.00","24,206,986.36","13,734,221.79",O


---
<h3>2. Identify the top 10 individual donors (i.e., persons) for each of the two presidential candidates and the amount they spent. In order to do that, you should know that donations are not always to a fundraising committee that can be directly linked to a candidate, but they can be due to other entities such as PACs.</h3>


---


* The first dataset, which will be used is the contributions from individuals for 2016. You can download both header file and data from https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv and https://www.fec.gov/files/bulk-downloads/2016/indiv16.zip respectively.

* The file contains contributor names, occupation and employer, address, and contribution amount.


In [43]:
indiv_headers = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv')
indiv_headers = indiv_headers.columns.tolist()
indiv_headers

['CMTE_ID',
 'AMNDT_IND',
 'RPT_TP',
 'TRANSACTION_PGI',
 'IMAGE_NUM',
 'TRANSACTION_TP',
 'ENTITY_TP',
 'NAME',
 'CITY',
 'STATE',
 'ZIP_CODE',
 'EMPLOYER',
 'OCCUPATION',
 'TRANSACTION_DT',
 'TRANSACTION_AMT',
 'OTHER_ID',
 'TRAN_ID',
 'FILE_NUM',
 'MEMO_CD',
 'MEMO_TEXT',
 'SUB_ID']

* Before we import the data, we should convert the field TRANSACTION_AMT to float and all the other fields to strings.

In [44]:
indiv_data_types = {header: np.str for header in indiv_headers}
indiv_data_types['TRANSACTION_AMT'] = np.float

* Full data of the file individuals can be read

In [45]:
individuals = pd.read_csv('C:/Users/tzina/Downloads/indiv16/itcont.txt', 
                  sep="|", 
                  index_col=False,
                  names=indiv_headers,
                  dtype=indiv_data_types,
                  parse_dates=['TRANSACTION_DT'])
print(individuals.shape)
individuals.head(5)

(20498303, 21)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00088591,N,M3,P,15970306895,15,IND,"BURCH, MARY K.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP PROGRAM MANAGEMENT,02132015,500.00,NaN,2A8EE0688413416FA735,998834,NaN,NaN,4032020151240885624
1,C00088591,N,M3,P,15970306960,15,IND,"KOUNTZ, DONALD E.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,DIR PROGRAMS,02132015,200.00,NaN,20150211113220-479,998834,NaN,NaN,4032020151240885819
2,C00088591,N,M3,P,15970306960,15,IND,"KOUNTZ, DONALD E.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,DIR PROGRAMS,02272015,200.00,NaN,20150225112333-476,998834,NaN,NaN,4032020151240885820
3,C00088591,N,M3,P,15970306915,15,IND,"DOSHI, NIMISH M.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP AND CFO,02132015,200.00,NaN,20150309_2943,998834,NaN,NaN,4032020151240885683
4,C00088591,N,M3,P,15970306915,15,IND,"DOSHI, NIMISH M.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP AND CFO,02272015,200.00,NaN,20150224153748-2525,998834,NaN,NaN,4032020151240885684


* Convert TRANSACTION_DT to datetime
* Remove TRANSACTION_DT = '1015-10-05' as we cannot parse the field

In [46]:
individuals = individuals[individuals['TRANSACTION_DT'] != '1015-10-05']

In [47]:
individuals['TRANSACTION_DT']= pd.to_datetime(individuals['TRANSACTION_DT'], format='%m%d%Y', errors = 'coerce')

* We will keep only the positive contributions.

In [48]:
individuals = individuals[individuals['TRANSACTION_AMT'] > 0]

* For committe master you can download both header file and data from https://www.fec.gov/data/browse-data/files/bulk-downloads/data_dictionaries/cm_header_file.csv and https://www.fec.gov/files/bulk-downloads/2016/cm16.zip respectively.


In [49]:
cm_headers =  pd.read_csv('https://www.fec.gov/data/browse-data/files/bulk-downloads/data_dictionaries/cm_header_file.csv')
cm_headers = cm_headers.columns.tolist()
cm_headers

['CMTE_ID',
 'CMTE_NM',
 'TRES_NM',
 'CMTE_ST1',
 'CMTE_ST2',
 'CMTE_CITY',
 'CMTE_ST',
 'CMTE_ZIP',
 'CMTE_DSGN',
 'CMTE_TP',
 'CMTE_PTY_AFFILIATION',
 'CMTE_FILING_FREQ',
 'ORG_TP',
 'CONNECTED_ORG_NM',
 'CAND_ID']

* We are ready to read the data.

In [50]:
committes = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/cm16.zip', 
                  sep="|", 
                  index_col=False,
                  names=cm_headers)
print(committes.shape)
committes.head(5)

(17651, 15)


,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,HALLMARK CARDS PAC,ERIN BROWER,2501 MCGEE,MD#288,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,20001,B,Q,NaN,M,M,AMERICAN MEDICAL ASSOCIATION,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,TOM RITTER,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,TEAMSTERS LOCAL UNION 886,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,"C. RICHARD BONEBRAKE, M.D.",623 SW 10TH AVE,NaN,TOPEKA,KS,66612,U,Q,UNK,Q,T,NaN,NaN
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,"VIDYA KORA, M.D.","322 CANAL WALK, CANAL LEVEL",NaN,INDIANAPOLIS,IN,46202,U,Q,NaN,Q,M,NaN,NaN


* Merge individuals with committes

In [52]:
fec = pd.merge(individuals, 
               committes[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], 
               on='CMTE_ID', 
               how='inner')
print(fec.shape)
fec.head(5)

(20114561, 23)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CMTE_NM,CAND_ID
0,C00088591,N,M3,P,15970306895,15,IND,"BURCH, MARY K.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP PROGRAM MANAGEMENT,2015-02-13,500.00,NaN,2A8EE0688413416FA735,998834,NaN,NaN,4032020151240885624,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
1,C00088591,N,M3,P,15970306960,15,IND,"KOUNTZ, DONALD E.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,DIR PROGRAMS,2015-02-13,200.00,NaN,20150211113220-479,998834,NaN,NaN,4032020151240885819,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
2,C00088591,N,M3,P,15970306960,15,IND,"KOUNTZ, DONALD E.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,DIR PROGRAMS,2015-02-27,200.00,NaN,20150225112333-476,998834,NaN,NaN,4032020151240885820,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
3,C00088591,N,M3,P,15970306915,15,IND,"DOSHI, NIMISH M.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP AND CFO,2015-02-13,200.00,NaN,20150309_2943,998834,NaN,NaN,4032020151240885683,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
4,C00088591,N,M3,P,15970306915,15,IND,"DOSHI, NIMISH M.",FALLS CHURCH,VA,220424511,NORTHROP GRUMMAN,VP AND CFO,2015-02-27,200.00,NaN,20150224153748-2525,998834,NaN,NaN,4032020151240885684,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN


* We should merge the fec dataset with candidates from previous question

In [53]:
fec = pd.merge(fec, 
               candidates[['CAND_ID', 'CAND_NAME']], 
               on='CAND_ID',
               how='inner')
print(fec.shape)
fec.head(5)

(2662041, 24)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CMTE_NM,CAND_ID,CAND_NAME
0,C00575795,A,Q2,P2016,201509039001600101,15,IND,"CLARY, CAROLE",DESTIN,FL,325405205,"CPR OF DESTIN, INC.",VP/DIRECTOR OF OPERATIONS,2015-06-15,50.00,NaN,C240090,1024052,NaN,NaN,4090920151249600958,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
1,C00575795,A,Q2,P2016,201509039001600101,15,IND,"FEW, MELANIE",DALLAS,TX,752011123,ACE CASH EXPRESS,MARKETING,2015-04-12,5.00,NaN,C17450,1024052,NaN,NaN,4090920151249600959,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
2,C00575795,A,Q2,P2016,201509039001600101,15,IND,"SPURLOCK, MICHAEL",NASHVILLE,TN,372036615,NaN,NOT EMPLOYED,2015-06-13,50.00,NaN,C234270,1024052,NaN,NaN,4090920151249600960,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
3,C00575795,A,Q2,P2016,201509039001600102,15,IND,"ELIAS, JOHN",WASHINGTON,DC,200053743,U.S. DEPARTMENT OF JUSTICE,ATTORNEY,2015-06-11,100.00,NaN,C223180,1024052,NaN,NaN,4090920151249600961,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
4,C00575795,A,Q2,P2016,201509039001600102,15,IND,"SHIFTON, CONSTANCE",CHERRY HILL,NJ,080031314,SELF-EMPLOYED,EDUCATIONAL ADVISOR,2015-06-12,50.00,NaN,C229360,1024052,NaN,NaN,4090920151249600962,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"


* Check the unique candidates

In [54]:
fec['CAND_NAME'].unique()

array(['CLINTON, HILLARY RODHAM', 'TRUMP, DONALD J.'], dtype=object)

In [55]:
fec['ENTITY_TP'].unique()

array(['IND', 'CAN', 'ORG', 'PAC', 'COM'], dtype=object)

* From ENTITY_TP keep only the indivinduals

In [56]:
fec = fec.loc[fec['ENTITY_TP'].isin(['IND'])]

* Create the dataframe for H.Clinton

In [57]:
f_clinton = fec.loc[fec['CAND_NAME'] == 'CLINTON, HILLARY RODHAM']
f_clinton = f_clinton[['CMTE_ID', 'TRANSACTION_TP', 'NAME', 'ENTITY_TP','TRANSACTION_AMT', 'CAND_NAME']]
print(f_clinton.shape)
f_clinton.head(5)

(2514936, 6)


,CMTE_ID,TRANSACTION_TP,NAME,ENTITY_TP,TRANSACTION_AMT,CAND_NAME
0,C00575795,15,"CLARY, CAROLE",IND,50.00,"CLINTON, HILLARY RODHAM"
1,C00575795,15,"FEW, MELANIE",IND,5.00,"CLINTON, HILLARY RODHAM"
2,C00575795,15,"SPURLOCK, MICHAEL",IND,50.00,"CLINTON, HILLARY RODHAM"
3,C00575795,15,"ELIAS, JOHN",IND,100.00,"CLINTON, HILLARY RODHAM"
4,C00575795,15,"SHIFTON, CONSTANCE",IND,50.00,"CLINTON, HILLARY RODHAM"


Rename field CAND_NAME

In [58]:
occ_mapping = {
    'TRUMP, DONALD J.': 'CLINTON, HILLARY RODHAM'
}

f = lambda x: occ_mapping.get(x, x) 
super_clinton.loc[:, 'CAND_NAME'] = super_clinton.loc[:, 'CAND_NAME'].map(f)

* Create the dataframe for D.Trump

In [59]:
f_trump = fec.loc[fec['CAND_NAME'] == 'TRUMP, DONALD J.']
f_trump = f_trump[['CMTE_ID', 'TRANSACTION_TP', 'NAME', 'ENTITY_TP','TRANSACTION_AMT', 'CAND_NAME']]
print(f_trump.shape)
f_trump.head(5)

(146846, 6)


,CMTE_ID,TRANSACTION_TP,NAME,ENTITY_TP,TRANSACTION_AMT,CAND_NAME
2515042,C00580100,15,"BONNETT, ELLIOTT",IND,60.00,"TRUMP, DONALD J."
2515043,C00580100,15,"FISHER, SANDI",IND,100.00,"TRUMP, DONALD J."
2515044,C00580100,15,"FLAHART, MICHAEL",IND,50.00,"TRUMP, DONALD J."
2515045,C00580100,15,"FLANAGAN, DAVID",IND,162.00,"TRUMP, DONALD J."
2515046,C00580100,15,"LANDES, PATRICIA",IND,10.00,"TRUMP, DONALD J."


Rename field CAND_NAME

In [60]:
occ_mapping = {
    'CLINTON, HILLARY RODHAM': 'TRUMP, DONALD J.'
}

f = lambda x: occ_mapping.get(x, x) 
super_trump.loc[:, 'CAND_NAME'] = super_trump.loc[:, 'CAND_NAME'].map(f)

* Merge the new dataset of H.Clinton with the dataset created in question 1 (including the transaction amounts of PACs and superPACs)

In [61]:
total_clinton = pd.concat([super_clinton, f_clinton])
print(total_clinton.shape)
total_clinton.sample(5)

(2558759, 10)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME,CMTE_ID,ENTITY_TP
969812,"CLINTON, HILLARY RODHAM",NaN,NaN,15,50.00,nan,nan,"KAPLAN, HOWARD",C00575795,IND
125773,"CLINTON, HILLARY RODHAM",NaN,NaN,15,25.00,nan,nan,"SOLOMON, ELIOT",C00575795,IND
1628359,"CLINTON, HILLARY RODHAM",NaN,NaN,15,38.00,nan,nan,"SPRIGGS, LUCY",C00575795,IND
2243373,"CLINTON, HILLARY RODHAM",NaN,NaN,15,38.00,nan,nan,"MCCORMICK, PATRICIA",C00575795,IND
1626003,"CLINTON, HILLARY RODHAM",NaN,NaN,15,100.00,nan,nan,"MCCULLOUGH, CATHERINE",C00575795,IND


* Merge the new dataset of D.Trump with the dataset created in question 1 (including the transaction amounts of PACs and superPACs)

In [62]:
total_trump = pd.concat([super_trump, f_trump])
print(total_trump.shape)
total_trump.sample(5)

(153530, 10)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME,CMTE_ID,ENTITY_TP
2539937,"TRUMP, DONALD J.",NaN,NaN,15,250.00,nan,nan,"BOYD, ANTHONY",C00580100,IND
2550618,"TRUMP, DONALD J.",NaN,NaN,15,58.00,nan,nan,"POWERS, ALLISON",C00580100,IND
2575606,"TRUMP, DONALD J.",NaN,NaN,15,146.00,nan,nan,"FLERCHINGER, JOHN",C00580100,IND
2640335,"TRUMP, DONALD J.",NaN,NaN,15,35.00,nan,nan,"BARDON, DAVID",C00580100,IND
2572874,"TRUMP, DONALD J.",NaN,NaN,15,250.00,nan,nan,"PAUL, CASTANEDO",C00580100,IND


<h4>The TOP 10 donors of Hillary Clinton </h4>

In [63]:
donors_clinton = total_clinton.groupby(['NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
donors_clinton.head(10)

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
NAME,,,
TARGETED PLATFORM MEDIA LLC,"99,211,787.00","6,598,946,725.53","4,592,783,612.14"
PRECISION NETWORK LLC,"18,551,753.00","6,340,310,089.57","4,404,039,072.65"
BLUEWEST MEDIA,"11,075,368.00","211,722,039.04","209,019,623.20"
WATERFRONT STRATEGIES,"7,819,657.00","3,805,367,191.84","2,879,275,710.91"
ADELSTEIN & ASSOCIATES,"6,807,764.00","73,057,250.00","69,454,192.50"
FUSE,"6,189,832.00","4,705,913,835.32","3,262,541,491.80"
GCW MEDIA SERVICES,"5,591,365.00","285,373,545.75","274,906,781.55"
"TARGET ENTERPRISES, LLC","5,260,498.00","1,938,055,013.12","1,774,196,156.48"
"RED SEA, LLC","4,557,817.00","159,494,415.76","153,459,377.52"


<h4>The TOP 10 donors Donald Trump</h4>

In [64]:
trump_donors = total_trump.groupby(['NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
trump_donors.head(10)

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
NAME,,,
MULTI MEDIA SERVICES CORPORATION,"18,980,619.00","614,029,440.44","515,257,518.10"
DEL CIELO MEDIA,"17,825,000.00","174,975,640.28","169,848,293.09"
STARBOARD STRATEGIC,"13,842,906.00",0.00,0.00
"STARBOARD STRATEGIC, INC.","8,936,541.00","561,368,896.40","500,271,928.26"
RAPID RESPONSE TELEVISION LLC,"5,868,879.00","1,948,262,654.92","1,508,339,175.86"
INFOCISION MANAGEMENT CORP,"5,202,469.00","3,772,561,325.79","2,071,131,597.46"
TANGIBLE MEDIA INC,"5,000,000.00","57,369,145.36","47,216,659.82"
TARGET ENTERPRISES LLC,"4,423,479.00","250,598,928.14","59,691,232.37"
FRONTLINE STRATEGIES & MEDIA,"4,316,381.00","286,845,726.80","236,083,299.10"


---
<h3>3. Investigate the chronological evolution of the contributions made to and the expenditures made by the campaigns</h3>

---

* For operating expenditures and disbursements you can download both header file and data from https://www.fec.gov/files/bulk-downloads/data_dictionaries/oppexp_header_file.csv and https://www.fec.gov/files/bulk-downloads/2016/oppexp16.zip respectively.

In [109]:
opper_headers = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/oppexp_header_file.csv')
opper_headers = opper_headers.columns.tolist()
opper_headers

['CMTE_ID',
 'AMNDT_IND',
 'RPT_YR',
 'RPT_TP',
 'IMAGE_NUM',
 'LINE_NUM',
 'FORM_TP_CD',
 'SCHED_TP_CD',
 'NAME',
 'CITY',
 'STATE',
 'ZIP_CODE',
 'TRANSACTION_DT',
 'TRANSACTION_AMT',
 'TRANSACTION_PGI',
 'PURPOSE',
 'CATEGORY',
 'CATEGORY_DESC',
 'MEMO_CD',
 'MEMO_TEXT',
 'ENTITY_TP',
 'SUB_ID',
 'FILE_NUM',
 'TRAN_ID',
 'BACK_REF_TRAN_ID']

* Convert type of TRANSACTION_AMT to float

In [110]:
opper_data_types = {header:np.str for header in opper_headers}
opper_data_types['TRANSACTION_AMT'] = np.float

* The full data of opperating_expenditures can be read

In [116]:
opper_exp = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/oppexp16.zip', 
                        sep="|", 
                        index_col=False,
                        names=opper_headers,
                        dtype=opper_data_types)
print(opper_exp.shape)
opper_exp.head(5)

(1749832, 25)


,CMTE_ID,AMNDT_IND,RPT_YR,RPT_TP,IMAGE_NUM,LINE_NUM,FORM_TP_CD,SCHED_TP_CD,NAME,CITY,STATE,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,TRANSACTION_PGI,PURPOSE,CATEGORY,CATEGORY_DESC,MEMO_CD,MEMO_TEXT,ENTITY_TP,SUB_ID,FILE_NUM,TRAN_ID,BACK_REF_TRAN_ID
0,C00415182,N,2015,Q1,15951142498,17,F3,SB,CHASE CARDMEMBER SERVICE,WILMINGTON,DE,198865153,02/21/2015,"3,301.24",P2016,CREDIT CARD PAYMENT,001,Administrative/Salary/Overhead Expenses,NaN,NaN,ORG,4041520151241882404,1002978,VN81E9TS8X8,NaN
1,C00415182,N,2015,Q1,15951142495,17,F3,SB,GOOGLE INC.,SAN FRANCISCO,CA,941390001,01/21/2015,56.42,P2016,INTERNET SERVICE,001,Administrative/Salary/Overhead Expenses,X,*,ORG,4041520151241882396,1002978,VN81E9TQB00,VN81E9TQAP1
2,C00415182,N,2015,Q1,15951142495,17,F3,SB,LINKEDIN CORPORATION,MOUNTAIN VIEW,CA,94043,01/21/2015,49.95,P2016,INTERNET SERVICE,001,Administrative/Salary/Overhead Expenses,X,*,ORG,4041520151241882397,1002978,VN81E9TQB76,VN81E9TQAP1
3,C00415182,N,2015,Q1,15951142496,17,F3,SB,NATIONBUILDER,LOS ANGELES,CA,900131155,01/21/2015,99.00,P2016,INTERNET SERVICE,001,Administrative/Salary/Overhead Expenses,X,*,ORG,4041520151241882398,1002978,VN81E9TS9X1,VN81E9TQAP1
4,C00415182,N,2015,Q1,15951142496,17,F3,SB,"NGP VAN, INC.",WASHINGTON,DC,20005,01/21/2015,"2,000.00",P2016,INTERNET SERVICE,001,Administrative/Salary/Overhead Expenses,X,*,ORG,4041520151241882399,1002978,VN81E9TQAX7,VN81E9TQAP1


* We will keep only positive contributions.

In [117]:
opper_exp = opper_exp.loc[opper_exp['TRANSACTION_AMT'] > 0]

* Merge the above dataset with committes, so as to have the name of each committe

In [118]:
opper_exp = pd.merge(opper_exp, 
                     committes[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], 
                     on='CMTE_ID', 
                     how='inner')
print(opper_exp.shape)
opper_exp.sample(5)

(1732174, 27)


,CMTE_ID,AMNDT_IND,RPT_YR,RPT_TP,IMAGE_NUM,LINE_NUM,FORM_TP_CD,SCHED_TP_CD,NAME,CITY,STATE,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,TRANSACTION_PGI,PURPOSE,CATEGORY,CATEGORY_DESC,MEMO_CD,MEMO_TEXT,ENTITY_TP,SUB_ID,FILE_NUM,TRAN_ID,BACK_REF_TRAN_ID,CMTE_NM,CAND_ID
33841,C00498121,N,2015,Q2,201507159000158893,17,F3,SB,AT&T,ATLANTA,GA,303485414,06/05/2015,150.00,NaN,TELEPHONE SERVICE,NaN,NaN,X,NaN,ORG,4071620151247283014,1015061,SB17.10849.1,SB17.10849,ROGER WILLIAMS FOR U S CONGRESS COMMITTEE,H2TX33040
602861,C00549477,A,2015,YE,201603239011948444,17,F3,SB,IMPACT MAILING SERVICE,CHARLOTTE,NC,28273,11/17/2015,"3,733.62",NaN,JFC DIRECT MARKETING,001,Administrative/Salary/Overhead Expenses,X,NaN,ORG,4032320161277043904,1057102,SB17.5056.0,SB17.5056,THE PITTENGER VICTORY FUND,H2NC09134
1388235,C00431601,N,2015,YE,201601299004770712,21B,F3X,SB,"TRILOGY INTERACTIVE, LLC",MOUNTAIN VIEW,CA,94040,08/03/2015,302.00,NaN,INTERNET HOSTING,NaN,NaN,NaN,NaN,ORG,4020220161261402807,1044862,D468404,NaN,OCEANS PAC,NaN
1092098,C00575795,A,2016,M7,201609189030921736,23,F3P,SB,EXPEDIA,BELLEVUE,WA,980045703,05/18/2016,168.37,P2016,TRAVEL,NaN,NaN,X,NaN,ORG,4092020161317302394,1099613,D234089,D215250,HILLARY FOR AMERICA,P00003392
643137,C00540500,N,2016,Q1,201604150200105945,17,F3,SB,BUFFER,SAN FRANCISCO,CA,941071872,01/28/2016,510.00,P,ONLINE ADVERTISEMENTS,NaN,NaN,X,MEMO ITEM,NaN,2042920161284602520,1066842,SB0427165945944,NaN,CORY BOOKER FOR SENATE,S4NJ00185


In [119]:
opper_exp = opper_exp[['CMTE_ID','NAME','TRANSACTION_DT','TRANSACTION_AMT','PURPOSE','CMTE_NM','STATE','CAND_ID']]
print(opper_exp.shape)
opper_exp.head(5)

(1732174, 8)


,CMTE_ID,NAME,TRANSACTION_DT,TRANSACTION_AMT,PURPOSE,CMTE_NM,STATE,CAND_ID
0,C00415182,CHASE CARDMEMBER SERVICE,02/21/2015,"3,301.24",CREDIT CARD PAYMENT,FRIENDS OF JOHN SARBANES,DE,H6MD03292
1,C00415182,GOOGLE INC.,01/21/2015,56.42,INTERNET SERVICE,FRIENDS OF JOHN SARBANES,CA,H6MD03292
2,C00415182,LINKEDIN CORPORATION,01/21/2015,49.95,INTERNET SERVICE,FRIENDS OF JOHN SARBANES,CA,H6MD03292
3,C00415182,NATIONBUILDER,01/21/2015,99.00,INTERNET SERVICE,FRIENDS OF JOHN SARBANES,CA,H6MD03292
4,C00415182,"NGP VAN, INC.",01/21/2015,"2,000.00",INTERNET SERVICE,FRIENDS OF JOHN SARBANES,DC,H6MD03292


* Merge the above merged dataset with candidates H.Clinton and D.Trump

In [120]:
opper_exp = pd.merge(opper_exp, 
                     candidates[['CAND_ID', 'CAND_NAME']], 
                     on='CAND_ID', 
                     how='inner')
print(opper_exp.shape)
opper_exp.sample(5)

(171866, 9)


,CMTE_ID,NAME,TRANSACTION_DT,TRANSACTION_AMT,PURPOSE,CMTE_NM,STATE,CAND_ID,CAND_NAME
127717,C00575795,TZELL TRAVEL GROUP,01/29/2016,10.00,TRAVEL,HILLARY FOR AMERICA,NY,P00003392,"CLINTON, HILLARY RODHAM"
146552,C00575795,AMERICAN EXPRESS,03/03/2016,30.00,TRAVEL,HILLARY FOR AMERICA,NJ,P00003392,"CLINTON, HILLARY RODHAM"
73901,C00575795,FEDEX,11/10/2016,220.57,EVENT SUPPLIES,HILLARY FOR AMERICA,TN,P00003392,"CLINTON, HILLARY RODHAM"
163002,C00575795,"KULES, AMANDA",06/09/2016,500.00,TRAVEL AND SUBSISTENCE,HILLARY FOR AMERICA,DC,P00003392,"CLINTON, HILLARY RODHAM"
125119,C00575795,AMERICAN EXPRESS,06/05/2016,9.00,TRAVEL,HILLARY FOR AMERICA,NJ,P00003392,"CLINTON, HILLARY RODHAM"


In [121]:
opper_exp['CAND_NAME'].unique()

array(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'], dtype=object)

* Convert TRANSACTION_DT field to datatime, so as to combine with the below

In [122]:
opper_exp['TRANSACTION_DT']= pd.to_datetime(opper_exp['TRANSACTION_DT'])

* Although the combine dataset with contributions from committes and candidates H.Clinton and D.Trump already exist, it has only TRANSACTIONS_TPs 24A and 24E. We will reproduce this dataset from the beginning

* Check for refunds and keep only the positive contributions

* Revert the type of TRANSACTION_DT to datetime

In [123]:
pas_contrib_new = pd.merge(pas_contrib, candidates[['CAND_ID', 'CAND_NAME']], on='CAND_ID', how='inner')
pas_contrib_new['TRANSACTION_DT'] = pd.to_datetime(pas_contrib_new['TRANSACTION_DT'])
print(pas_contrib_new.shape)
pas_contrib_new.sample(5)

(111409, 23)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
49441,C90011156,N,YE,G2016,201701319042181626,24E,IND,"POND, NATHANIEL",PORTLAND,OR,972363342,NaN,NaN,2016-11-05,49.00,P00003392,P00003392,VN7CZA7EPM0,1144686,NaN,NaN,4020920171370173618,"CLINTON, HILLARY RODHAM"
33868,C90011156,N,YE,G2016,201701319042153164,24E,ORG,BUDGET-GREENSBORO,GREENSBORO,NC,274074619,NaN,NaN,2016-09-10,7.00,P00003392,P00003392,VN7CZA37ZN2,1144686,NaN,NaN,4020920171370088233,"CLINTON, HILLARY RODHAM"
106525,C90011156,N,YE,G2016,201701319042167084,24A,IND,"FREDMAN, JOSHUA",SEATTLE,WA,981051745,NaN,NaN,2016-10-13,4.00,P80001571,P80001571,VN7CZA5AQX4,1144686,NaN,NaN,4020920171370129991,"TRUMP, DONALD J."
48405,C90011156,N,YE,G2016,201701319042177928,24E,IND,"WALKER, AMBER",PITTSBURGH,PA,152182527,NaN,NaN,2016-10-31,21.00,P00003392,P00003392,VN7CZA6XB95,1144686,NaN,NaN,4020920171370162524,"CLINTON, HILLARY RODHAM"
44003,C90011156,N,YE,G2016,201701319042142134,24E,IND,"REPPERT, TARAH",PHILADELPHIA,PA,191191508,NaN,NaN,2016-07-18,89.00,P00003392,P00003392,VN7CZA1RQ34,1144686,NaN,NaN,4020920171370055143,"CLINTON, HILLARY RODHAM"


In [124]:
(pas_contrib_new['TRANSACTION_AMT']>0).value_counts()

True     111024
False       385
Name: TRANSACTION_AMT, dtype: int64

In [125]:
pas_contrib_new = pas_contrib_new.loc[pas_contrib_new['TRANSACTION_AMT'] > 0]

In [126]:
pas_contrib_new['TRANSACTION_TP'].unique()

array(['24K', '24A', '24E', '24Z', '24F', '24C', '24N'], dtype=object)

Let's have a brief summary for the above TRANSACTION_TPs from https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/

* 24K: Contribution made to nonaffiliated committee
* 24A: Independent expenditure opposing election of candidate
* 24E: Independent expenditure advocating election of candidate
* 24Z: In-kind contribution made to registered filer
* 24F: Communication cost for candidate (only for Form 7 filer)
* 24C: Coordinated party expenditure
* 24N: Communication cost against candidate (only for Form 7 filer)

The types 24Z and 24C will be executed as they are irrelevant with money.

In [127]:
pas_contrib_new = pas_contrib_new[pas_contrib_new['TRANSACTION_TP'].isin(['24K', '24A', '24E', '24F', '24N'])]
print(pas_contrib_new.shape)
pas_contrib_new.sample(5)

(110795, 23)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
87836,C90011156,N,YE,G2016,201701319042174616,24A,IND,"TOTTEN, ALI",REYNOLDSBURG,OH,430684253,NaN,NaN,2016-10-26,34.00,P80001571,P80001571,VN7CZA6EFR4,1144686,NaN,NaN,4020920171370152587,"TRUMP, DONALD J."
36277,C90011156,N,YE,G2016,201701319042149509,24E,IND,"MCCAIN, HOPE",CLEVELAND,OH,441102525,NaN,NaN,2016-08-30,34.00,P00003392,P00003392,VN7CZA2RXY1,1144686,NaN,NaN,4020920171370077268,"CLINTON, HILLARY RODHAM"
30325,C90011156,N,YE,G2016,201701319042160264,24E,IND,"HAMILTON, MAURICE",DURHAM,NC,277034448,NaN,NaN,2016-09-28,17.00,P00003392,P00003392,VN7CZA45X00,1144686,NaN,NaN,4020920171370109533,"CLINTON, HILLARY RODHAM"
38611,C90011156,N,YE,G2016,201701319042142447,24E,IND,"GARRETT, DEANNA",FORT THOMAS,KY,410752520,NaN,NaN,2016-07-21,27.00,P00003392,P00003392,VN7CZA1TXQ0,1144686,NaN,NaN,4020920171370056082,"CLINTON, HILLARY RODHAM"
93787,C90011156,N,YE,G2016,201701319042146191,24A,IND,"ADAMS, CHRISTIAN",CINCINNATI,OH,452202216,NaN,NaN,2016-08-17,34.00,P80001571,P80001571,VN7CZA2CFD3,1144686,NaN,NaN,4020920171370067314,"TRUMP, DONALD J."


In [128]:
pas_contrib_new['CAND_NAME'].unique()

array(['CLINTON, HILLARY RODHAM', 'TRUMP, DONALD J.'], dtype=object)

We will remove some columns from our dataset

In [129]:
pas_contrib_new = pas_contrib_new[['CAND_ID','CAND_NAME','CMTE_ID','TRANSACTION_TP','ENTITY_TP','TRANSACTION_DT','TRANSACTION_AMT']]
print(pas_contrib_new.shape)
pas_contrib_new.head(5)

(110795, 7)


,CAND_ID,CAND_NAME,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT
0,P00003392,"CLINTON, HILLARY RODHAM",C00553966,24K,COM,2015-06-11,"2,700.00"
1,P00003392,"CLINTON, HILLARY RODHAM",C00574228,24K,CCM,2015-04-16,500.00
2,P00003392,"CLINTON, HILLARY RODHAM",C00574228,24K,CCM,2015-05-06,"2,200.00"
3,P00003392,"CLINTON, HILLARY RODHAM",C00540955,24K,CCM,2015-06-22,"5,000.00"
4,P00003392,"CLINTON, HILLARY RODHAM",C00342048,24K,CCM,2015-05-05,"5,000.00"


A new combined data frame will be created, including commites inside the pas_contrib_new

In [130]:
pas_contrib_new = pd.merge(pas_contrib_new, 
                           committes[['CMTE_ID', 'CMTE_NM']], 
                           on='CMTE_ID', 
                           how='inner')
print(pas_contrib_new.shape)
pas_contrib_new.head(5)

(110795, 8)


,CAND_ID,CAND_NAME,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM
0,P00003392,"CLINTON, HILLARY RODHAM",C00553966,24K,COM,2015-06-11,"2,700.00","YELP, INC. POLITICAL ACTION COMMITTEE"
1,P00003392,"CLINTON, HILLARY RODHAM",C00574228,24K,CCM,2015-04-16,500.00,CINCINNATUS PAC
2,P00003392,"CLINTON, HILLARY RODHAM",C00574228,24K,CCM,2015-05-06,"2,200.00",CINCINNATUS PAC
3,P00003392,"CLINTON, HILLARY RODHAM",C00574228,24K,CCM,2016-03-10,"2,300.00",CINCINNATUS PAC
4,P00003392,"CLINTON, HILLARY RODHAM",C00540955,24K,CCM,2015-06-22,"5,000.00",FEARLESS PAC


In [131]:
individuals['TRANSACTION_TP'].unique()

array(['15', '15E', '22Y', '11', '15C', '24T', '24I', '10', '31', '20Y',
       '32', '30', '31T', '21Y', '32T', '19', '30T', '41T', '40Y'],
      dtype=object)

In [132]:
individuals_new = individuals[['CMTE_ID', 'TRANSACTION_TP', 'ENTITY_TP', 'TRANSACTION_DT', 'TRANSACTION_AMT']]

In [133]:
individuals_new = pd.merge(individuals_new, 
                           committes[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], 
                           on='CMTE_ID', 
                           how='inner')
print(individuals_new.shape)
individuals_new.sample(5)

(20114561, 7)


,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM,CAND_ID
13692152,C00578401,15,IND,2016-06-28,500.00,FRIENDS OF ANNA THRONE-HOLST,H6NY01126
3420262,C00401224,24T,IND,2016-04-15,35.00,ACTBLUE,NaN
2904390,C00401224,24T,IND,2015-02-28,50.00,ACTBLUE,NaN
516742,C00574624,15,IND,2015-12-30,200.00,CRUZ FOR PRESIDENT,P60006111
13549497,C00108035,15,IND,2016-10-14,30.00,MCKESSON CORPORATION EMPLOYEES POLITICAL FUND,NaN


* Remove some TRANSACTION_TPs irrelevant to the contributions regarding https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/

In [134]:
individuals_tr = individuals_new.loc[individuals_new['TRANSACTION_TP'].isin(['15', '15E', '11', '15C', '24T', '24I', '10', '32', '30', '32T',
                                                       '19', '30T'])].copy()
print(individuals_tr.shape)
individuals_tr.head(5)

(19878533, 7)


,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM,CAND_ID
0,C00088591,15,IND,2015-02-13,500.00,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
1,C00088591,15,IND,2015-02-13,200.00,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
2,C00088591,15,IND,2015-02-27,200.00,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
3,C00088591,15,IND,2015-02-13,200.00,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN
4,C00088591,15,IND,2015-02-27,200.00,EMPLOYEES OF NORTHROP GRUMMAN CORPORATION PAC,NaN


* Keep this file with candidates, to eliminate our analysis to H.Clinton and D.Trump

In [135]:
individuals_tr = pd.merge(individuals_tr, 
                          candidates[['CAND_ID', 'CAND_NAME']], 
                          on='CAND_ID', 
                          how='inner')
print(individuals_tr.shape)
individuals_tr.sample(5)

(2629716, 8)


,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM,CAND_ID,CAND_NAME
44818,C00575795,15,IND,2015-09-21,500.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
256635,C00575795,15,IND,2016-10-28,1.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
1646995,C00575795,15,IND,2016-08-26,25.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
27517,C00575795,15,IND,2015-06-25,"2,700.00",HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
291169,C00575795,15,IND,2016-10-21,25.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"


In [136]:
individuals_tr['CAND_NAME'].unique()

array(['CLINTON, HILLARY RODHAM', 'TRUMP, DONALD J.'], dtype=object)

After calculating all the above, we have to split them in different dataset regarding the candidate

Hillary Clinton's datasets

<h4>Group Clinton's pas contributions per month </h4>

In [137]:
pas_contrib_clinton = pas_contrib_new.loc[pas_contrib_new['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(pas_contrib_clinton.shape)
pas_contrib_clinton.sample(5)

(65326, 8)


,CAND_ID,CAND_NAME,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM
11278,P00003392,"CLINTON, HILLARY RODHAM",C90011156,24E,IND,2016-09-17,61.00,WORKING AMERICA
38431,P00003392,"CLINTON, HILLARY RODHAM",C90011156,24E,IND,2016-07-22,34.00,WORKING AMERICA
59445,P00003392,"CLINTON, HILLARY RODHAM",C90011156,24E,IND,2016-10-05,34.00,WORKING AMERICA
47681,P00003392,"CLINTON, HILLARY RODHAM",C90011156,24E,IND,2016-11-01,72.00,WORKING AMERICA
23425,P00003392,"CLINTON, HILLARY RODHAM",C90011156,24E,IND,2016-09-29,17.00,WORKING AMERICA


In [138]:
pas_contrib_clinton['TRANSACTION_DT'] = pd.to_datetime(pas_contrib_clinton['TRANSACTION_DT'], format='%m%d%Y')
pas_contrib_clinton_group = pas_contrib_clinton.groupby([(pas_contrib_clinton['TRANSACTION_DT'].dt.to_period('M'))]).sum()
pas_contrib_clinton_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-01,661.00
2015-02,250.00
2015-03,"42,800.00"
2015-04,"172,085.00"
2015-05,"333,944.00"
2015-06,"1,174,967.00"
2015-07,"492,504.00"
2015-08,"518,764.00"
2015-09,"262,914.00"


* Remove the dates that are not related to period 2015-2016

In [139]:
pas_contrib_clinton_group[:-3]
pas_contrib_clinton_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-01,661.00
2015-02,250.00
2015-03,"42,800.00"
2015-04,"172,085.00"
2015-05,"333,944.00"
2015-06,"1,174,967.00"
2015-07,"492,504.00"
2015-08,"518,764.00"
2015-09,"262,914.00"


<h4>Group Clinton's indivinduals contributions per month </h4>

In [140]:
individuals_tr_clinton = individuals_tr.loc[individuals_tr['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(individuals_tr_clinton.shape)
individuals_tr_clinton.sample(5)

(2483565, 8)


,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM,CAND_ID,CAND_NAME
463320,C00575795,15,IND,2016-10-22,10.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
1296340,C00575795,15,IND,2016-10-08,37.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
2199339,C00575795,15,IND,2016-04-19,25.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
9689,C00575795,15,IND,2015-06-23,121.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"
1831696,C00575795,15,IND,2016-06-28,25.00,HILLARY FOR AMERICA,P00003392,"CLINTON, HILLARY RODHAM"


In [141]:
individuals_tr_clinton['TRANSACTION_DT'] = pd.to_datetime(individuals_tr_clinton['TRANSACTION_DT'], format='%m%d%Y')
individuals_tr_clinton_group = individuals_tr_clinton.groupby([(individuals_tr_clinton['TRANSACTION_DT'].dt.to_period('M'))]).sum()
individuals_tr_clinton_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-04,"13,358,043.00"
2015-05,"11,946,341.00"
2015-06,"13,831,957.00"
2015-07,"6,992,381.00"
2015-08,"5,812,265.00"
2015-09,"9,696,825.00"
2015-10,"7,766,163.00"
2015-11,"8,747,813.00"
2015-12,"8,306,308.00"


<h4>Group Clinton's opperating expenditures per month </h4>

In [142]:
opper_exp_clinton = opper_exp.loc[opper_exp['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(opper_exp_clinton.shape)
opper_exp_clinton.sample(5)

(141664, 9)


,CMTE_ID,NAME,TRANSACTION_DT,TRANSACTION_AMT,PURPOSE,CMTE_NM,STATE,CAND_ID,CAND_NAME
33474,C00575795,"BLANDIN, NEISHA",2015-05-15,"1,726.58",PAYROLL,HILLARY FOR AMERICA,NY,P00003392,"CLINTON, HILLARY RODHAM"
60606,C00575795,"PAUL, MATTHEW",2016-08-15,"3,815.91",PAYROLL,HILLARY FOR AMERICA,NY,P00003392,"CLINTON, HILLARY RODHAM"
126292,C00575795,UNITED AIRLINES,2016-06-09,470.25,TRAVEL,HILLARY FOR AMERICA,IL,P00003392,"CLINTON, HILLARY RODHAM"
39842,C00575795,COACH USA,2015-06-30,34.25,TRAVEL,HILLARY FOR AMERICA,NY,P00003392,"CLINTON, HILLARY RODHAM"
42171,C00575795,"SOWAH, HELENA-JASMINE",2015-09-15,"1,441.03",PAYROLL,HILLARY FOR AMERICA,NY,P00003392,"CLINTON, HILLARY RODHAM"


In [143]:
opper_exp_clinton['TRANSACTION_DT'] = pd.to_datetime(opper_exp_clinton['TRANSACTION_DT'], format='%m%d%Y')
opper_exp_clinton_group = opper_exp_clinton.groupby([(opper_exp_clinton['TRANSACTION_DT'].dt.to_period('M'))]).sum()
opper_exp_clinton_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-01,"41,059.40"
2015-02,"38,707.07"
2015-03,"93,299.70"
2015-04,"3,228,321.12"
2015-05,"7,261,633.18"
2015-06,"9,158,569.12"
2015-07,"9,466,236.63"
2015-08,"9,270,133.15"
2015-09,"7,789,361.79"


  * Donald Trump's datasets

<h4>Group Trump's pas contributions per month </h4>

In [144]:
pas_contrib_trump = pas_contrib_new.loc[pas_contrib_new['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(pas_contrib_trump.shape)
pas_contrib_trump.sample(5)

(45469, 8)


,CAND_ID,CAND_NAME,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM
91411,P80001571,"TRUMP, DONALD J.",C90011156,24A,IND,2016-10-25,22.00,WORKING AMERICA
93581,P80001571,"TRUMP, DONALD J.",C90011156,24A,IND,2016-11-01,17.00,WORKING AMERICA
5074,P80001571,"TRUMP, DONALD J.",C00508440,24A,ORG,2016-10-30,"8,932.00",HUMAN RIGHTS CAMPAIGN EQUALITY VOTES
103920,P80001571,"TRUMP, DONALD J.",C90011156,24A,ORG,2016-10-05,11.00,WORKING AMERICA
110033,P80001571,"TRUMP, DONALD J.",C00521013,24A,IND,2016-04-29,225.00,FLORIDA FREEDOM PAC


In [145]:
pas_contrib_trump['TRANSACTION_DT'] = pd.to_datetime(pas_contrib_trump['TRANSACTION_DT'], format='%m%d%Y')
pas_contrib_trump_group = pas_contrib_trump.groupby([(pas_contrib_trump['TRANSACTION_DT'].dt.to_period('M'))]).sum()
pas_contrib_trump_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-06,272.00
2015-07,"27,300.00"
2015-08,"38,830.00"
2015-09,"1,153,912.00"
2015-10,"192,606.00"
2015-11,"148,296.00"
2015-12,"161,409.00"
2016-01,"3,671,641.00"
2016-02,"8,494,964.00"


<h4>Group Trump's indivinduals contributions per month </h4>

In [146]:
individuals_tr_trump = individuals_tr.loc[individuals_tr['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(individuals_tr_trump.shape)
individuals_tr_trump.sample(5)

(146151, 8)


,CMTE_ID,TRANSACTION_TP,ENTITY_TP,TRANSACTION_DT,TRANSACTION_AMT,CMTE_NM,CAND_ID,CAND_NAME
2509252,C00580100,15,IND,2016-06-22,250.00,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,"TRUMP, DONALD J."
2581100,C00580100,15,IND,2016-10-27,500.00,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,"TRUMP, DONALD J."
2492468,C00580100,15,IND,2016-06-22,250.00,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,"TRUMP, DONALD J."
2562393,C00580100,15,IND,2016-08-14,250.00,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,"TRUMP, DONALD J."
2604727,C00580100,15,IND,2016-07-28,200.00,"DONALD J. TRUMP FOR PRESIDENT, INC.",P80001571,"TRUMP, DONALD J."


In [147]:
individuals_tr_trump['TRANSACTION_DT'] = pd.to_datetime(individuals_tr_trump['TRANSACTION_DT'], format='%m%d%Y')
individuals_tr_trump_group = individuals_tr_trump.groupby([(individuals_tr_trump['TRANSACTION_DT'].dt.to_period('M'))]).sum()
individuals_tr_trump_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-06,"65,224.00"
2015-07,"264,742.00"
2015-08,"630,093.00"
2015-09,"259,451.00"
2015-10,"234,268.00"
2015-11,"160,560.00"
2015-12,"284,979.00"
2016-01,"268,397.00"
2016-02,"492,956.00"


<h4>Group Trump's opperating expenditures per month </h4>

In [148]:
opper_exp_trump = opper_exp.loc[opper_exp['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(opper_exp_trump.shape)
opper_exp_trump.sample(5)

(30202, 9)


,CMTE_ID,NAME,TRANSACTION_DT,TRANSACTION_AMT,PURPOSE,CMTE_NM,STATE,CAND_ID,CAND_NAME
18640,C00580100,HOTELS.COM,2016-08-28,"1,274.40",TRAVEL: LODGING [AMEX: SB23.4584],"DONALD J. TRUMP FOR PRESIDENT, INC.",TX,P80001571,"TRUMP, DONALD J."
3008,C00580100,UBER,2015-07-31,51.98,TRAVEL: GROUND TRANSPORTATION [AMEX: SB23.6877],"DONALD J. TRUMP FOR PRESIDENT, INC.",CA,P80001571,"TRUMP, DONALD J."
1969,C00580100,UBER,2015-10-02,71.15,TRAVEL: GROUND TRANSPORTATION [AMEX: SB23.248391],"DONALD J. TRUMP FOR PRESIDENT, INC.",CA,P80001571,"TRUMP, DONALD J."
3575,C00580100,USPS,2015-09-03,21.66,DELIVERY SERVICES [MCENTEE: SB23.6733],"DONALD J. TRUMP FOR PRESIDENT, INC.",DC,P80001571,"TRUMP, DONALD J."
27668,C00580100,"PLAYFORTH, TAYLOR",2016-11-21,"2,666.38",TRAVEL EXPENSE REIMBURSEMENT: ITEMIZATION BELO...,"DONALD J. TRUMP FOR PRESIDENT, INC.",NY,P80001571,"TRUMP, DONALD J."


In [149]:
opper_exp_trump['TRANSACTION_DT'] = pd.to_datetime(opper_exp_trump['TRANSACTION_DT'], format='%m%d%Y')
opper_exp_trump_group = opper_exp_trump.groupby([(opper_exp_trump['TRANSACTION_DT'].dt.to_period('M'))]).sum()
opper_exp_trump_group

,TRANSACTION_AMT
TRANSACTION_DT,
2015-04,"197,738.32"
2015-05,"358,715.09"
2015-06,"870,798.78"
2015-07,"763,324.82"
2015-08,"1,850,246.51"
2015-09,"1,845,830.99"
2015-10,"1,691,236.99"
2015-11,"2,620,480.95"
2015-12,"3,076,835.53"


---
<h3>4. Identify the biggest recipients of campaign expenditures</h3>

---

Make sure before implementations, that file opperating expenditures is filtered for candidates H.Clinton and D.Trump

In [150]:
opper_exp['CAND_NAME'].unique()

array(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'], dtype=object)

Regarding the documentation https://www.fec.gov/campaign-finance-data/operating-expenditures-file-description/, we will group the data by candidate's name and name of Contributor/Lender/Transfer Name

</h4> The biggest recipients of campaign expenditures for Hillary Clinton</h4>

In [155]:
recipients_clinton = opper_exp[opper_exp['CAND_NAME'] == 'CLINTON, HILLARY RODHAM'].groupby(['CAND_NAME', 'NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
recipients_clinton.head(10)

TRANSACTION_AMT
CAND_NAME               NAME                                         
CLINTON, HILLARY RODHAM GMMB                           308,463,389.36
                        BULLY PULPIT INTERACTIVE LLC    33,306,462.95
                        ADP                             27,902,805.45
                        AMERICAN EXPRESS                23,977,194.86
                        MARKHAM PRODUCTIONS             16,025,851.54
                        EXECUTIVE FLITEWAYS             15,866,289.31
                        AIR PARTNERS, INC.              11,156,326.38
                        STRIPE                          10,199,840.66
                        AETNA                            5,883,908.50
                        MISSION CONTROL, INC.            5,837,599.86

</h4> The biggest recipients of campaign expenditures for Donald Trump</h4>

In [156]:
recipients_trump = opper_exp[opper_exp['CAND_NAME'] == 'TRUMP, DONALD J.'].groupby(['CAND_NAME', 'NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
recipients_trump.head(10)

TRANSACTION_AMT
CAND_NAME        NAME                                            
TRUMP, DONALD J. GILES-PARSCALE                     87,838,378.10
                 AMERICAN MEDIA & ADVOCACY GROUP    74,176,379.30
                 RICK REED MEDIA, INC.              24,359,873.08
                 AMERICAN EXPRESS                   19,393,367.83
                 ACE SPECIALTIES, LLC               15,885,323.63
                 PRIVATE JET SERVICES, LLC           9,953,216.06
                 JAMESTOWN ASSOCIATES                8,836,175.01
                 TAG AIR, INC.                       8,741,464.25
                 AIR CHARTER TEAM                    8,321,403.26
                 CAMBRIDGE ANALYTICA, LLC            5,912,500.00

---
<h3>5. Examine the geographical distribution, at the state level, of campaign expenditures. For each state, calculate the expenditures per voter. This will require that you find a source with the number of registered voters per state. Examine the situation for swing states.</h3>

---

* We will attempt to make a map of the US showing the donations by candidate per state.
* From file opperating expenditures group the data by candidate's name and state


In [157]:
states_group = opper_exp.groupby(['CAND_NAME', 'STATE'])
states_totals = states_group.sum().unstack(0).fillna(0)
states_totals.head(5)

TRANSACTION_AMT                 
CAND_NAME CLINTON, HILLARY RODHAM TRUMP, DONALD J.
STATE                                             
AK                      27,981.93        20,551.37
AL                   2,591,329.75       480,833.44
AR                  16,375,895.98       170,702.77
AZ                     231,822.41     3,390,953.43
BC                       4,432.96             0.00

* The data will be normalized, so that we get the percentages.

In [158]:
states_norm = states_totals.div(states_totals.sum(axis=1), axis=0)
states_norm.head(5)

TRANSACTION_AMT                 
CAND_NAME CLINTON, HILLARY RODHAM TRUMP, DONALD J.
STATE                                             
AK                           0.58             0.42
AL                           0.84             0.16
AR                           0.99             0.01
AZ                           0.06             0.94
BC                           1.00             0.00

* We will use a dictionary with US states from https://gist.github.com/mGalarnyk/27c99c5f55133a2ceb42b8c9a450d794

In [174]:
states = {
    'Alaska': 'AK',
    'Alabama': 'AL',
    'Arkansas': 'AR',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Iowa': 'IA',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Massachusetts': 'MA',
    'Maryland': 'MD',
    'Maine': 'ME',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Missouri': 'MO',
    'Northern Mariana Islands': 'MP',
    'Mississippi': 'MS',
    'Montana': 'MT',
    'National': 'NA',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Nebraska': 'NE',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'Nevada': 'NV',
    'New York': 'NY',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Virginia': 'VA',
    'Virgin Islands': 'VI',
    'Vermont': 'VT',
    'Washington': 'WA',
    'Wisconsin': 'WI',
    'West Virginia': 'WV',
    'Wyoming': 'WY',
    'Virgin Islands': 'VI'
}

Get the voters from the https://www.fec.gov/documents/1890/federalelections2016.xlsx

In [212]:
election_results = pd.read_excel(
    'https://www.fec.gov/documents/1890/federalelections2016.xlsx',
    sheet_name='Table 2. Electoral &  Pop Vote',
    skiprows=2,
    skipfooter=7,
    header=[0, 1])

In [213]:
print(election_results.shape)
election_results.head()

(51, 7)


STATE ELECTORAL VOTE             POPULAR VOTE              \
  Unnamed: 0_level_1      Trump (R) Clinton (D)    Trump (R) Clinton (D)   
0                 AL              9         NaN      1318255      729547   
1                 AK              3         NaN       163387      116454   
2                 AZ             11         NaN      1252401     1161167   
3                 AR              6         NaN       684872      380494   
4                 CA            NaN          55      4483814     8753792   

                         
  All Others Total Vote  
0      75570    2123372  
1      38767     318608  
2     159597    2573165  
3      65310    1130676  
4     943998   14181604

In [214]:
second_col_level = sorted(set(election_results.columns.get_level_values(level=1).tolist()))
print(second_col_level)
second_col_level[-1] = 'STATE'
print(second_col_level)
election_results.columns.set_levels(
    second_col_level,
    inplace=True,
    level=1)

['All Others', 'Clinton (D)', 'Total Vote', 'Trump (R)', 'Unnamed: 0_level_1']
['All Others', 'Clinton (D)', 'Total Vote', 'Trump (R)', 'STATE']


In [215]:
election_results.head()

STATE ELECTORAL VOTE             POPULAR VOTE                         \
  STATE      Trump (R) Clinton (D)    Trump (R) Clinton (D) All Others   
0    AL              9         NaN      1318255      729547      75570   
1    AK              3         NaN       163387      116454      38767   
2    AZ             11         NaN      1252401     1161167     159597   
3    AR              6         NaN       684872      380494      65310   
4    CA            NaN          55      4483814     8753792     943998   

              
  Total Vote  
0    2123372  
1     318608  
2    2573165  
3    1130676  
4   14181604

In [216]:
election_results['Winning Party'] = np.where(election_results[('ELECTORAL VOTE', 'Trump (R)')].isna(),'blue','red')
election_results.head()

STATE ELECTORAL VOTE             POPULAR VOTE                         \
  STATE      Trump (R) Clinton (D)    Trump (R) Clinton (D) All Others   
0    AL              9         NaN      1318255      729547      75570   
1    AK              3         NaN       163387      116454      38767   
2    AZ             11         NaN      1252401     1161167     159597   
3    AR              6         NaN       684872      380494      65310   
4    CA            NaN          55      4483814     8753792     943998   

             Winning Party  
  Total Vote                
0    2123372           red  
1     318608           red  
2    2573165           red  
3    1130676           red  
4   14181604          blue

We should adjust the column 'Total Vote' per candidate at the dataframe opperating expenditures.